In [1]:
# external modules
import sys
import random
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.dates import DateFormatter
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm

# Pandas setting
# Display 6 columns for viewing purposes
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# read temporary processed data
df = pd.read_csv("./../data/processed/tse1_unitchanged.csv", index_col=[0, 1, 2])
df_2008 = pd.read_csv("../data/processed/tse1_2008_balanced.csv", index_col=[0, 1, 2])

## 欠損処理

In [3]:
# # Visualize Missing Value in df_2008
# fig, ax = plt.subplots(figsize=(10, 100))
# ax = sns.heatmap(df_2008.isna(), cmap="binary", linewidths=0, linecolor="gray")
# ax.set_title("Missing Values in Original Dataset (all firm)")
# plt.show()

* 資本的支出と期末従業員数

四半期データは2008年に四半期開始、2011に簡素化されてる。(簡素化後、3ヶ月は任意。累積は義務)  
https://www.asb.or.jp/jp/accounting_standards/accounting_standards/y2007/2007-0314.html  
https://www.asb.or.jp/jp/accounting_standards/accounting_standards/y2011/2011-0325.html  
簡素化に伴い、資本的支出(CF計算書)、期末従業員数の報告が四半期で報告されなくなる。
--> Zhang et al. (2004)に倣い、資本的支出は4等分、期末従業員数は一定とする。

In [4]:
# proportion of missing value
df_2008[["資本的支出", "期末従業員数"]].isna().mean()

資本的支出    0.30
期末従業員数   0.54
dtype: float64

In [5]:
df_2008[["資本的支出", "期末従業員数"]].describe()

,資本的支出,期末従業員数
count,"51,955.00","34,556.00"
mean,"18,533,372,129.73","9,439.12"
std,"108,863,616,446.61","30,128.37"
min,"1,000,000.00",13.00
25%,"411,000,000.00",730.00
50%,"1,639,000,000.00","1,849.00"
75%,"6,891,000,000.00","5,690.25"
max,"5,121,960,000,000.00","385,243.00"


* 期末従業員数

欠損は前四半期時点の従業員数を代入する。(従業員数は変化していないという仮定)

In [6]:
df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"].isna().sum()

52

In [7]:
df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"][
    df_2008.loc[pd.IndexSlice[:, 2008, 6], :]["期末従業員数"].isna()
]

企業名              決算期（年）  決算期（月）
ひらまつ             2008    6        nan
アルインコ            2008    6        nan
エムティーアイ          2008    6        nan
カワチ薬品            2008    6        nan
キャリアデザインセンター     2008    6        nan
キーエンス            2008    6        nan
グリムス             2008    6        nan
コナカ              2008    6        nan
サイバーエージェント       2008    6        nan
システムソフト          2008    6        nan
シミックホールディングス     2008    6        nan
ジャパンベストレスキューシステ  2008    6        nan
ステップ             2008    6        nan
ティア              2008    6        nan
ディア・ライフ          2008    6        nan
ネクシィーズグループ       2008    6        nan
パラカ              2008    6        nan
ファルコホールディングス     2008    6        nan
フルキャストホールディングス   2008    6        nan
ホソカワミクロン         2008    6        nan
メディカルシステムネットワーク  2008    6        nan
ヤマウラ             2008    6        nan
ヤマト              2008    6        nan
ユニカフェ            2008    6        nan
ヨコレイ             2008    6        nan
ランドビジネス          2

最初の期2008Q1が欠損してしまっている企業が 52 / 1436 ある。df_2008だけでは欠損値補填ができないのでdfに戻ってさらにさかのぼって補填

In [8]:
year_range = np.arange(2008, 2021)
month_range = [6, 9, 12, 3]

# check if firm recorded in all quater
# start from firm recorded in 2008Q1
recorded_firm_from_2008 = set(df.loc[pd.IndexSlice[:, 2008, 6], :].index.get_level_values("企業名").unique())

for i in year_range:
    for j in month_range:
        if j == 3: # if Mar, +1 year
            i += 1
        recorded_firm_from_2008 = recorded_firm_from_2008 & set(df.loc[pd.IndexSlice[:, i, j], :].index.get_level_values("企業名").unique())

df_2008 = df.loc[pd.IndexSlice[recorded_firm_from_2008, :, :], :] # 遡るためrecorded_firm_from_2008の企業の全期間 : のデータフレームをもう一度作成

In [9]:
df_2008.loc[pd.IndexSlice[["アサヒグループホールディングス", "アシックス"], :, :], :]["期末従業員数"].iloc[70:90]

企業名              決算期（年）  決算期（月）
アサヒグループホールディングス  2018    6              nan
                         9              nan
                         12       28,055.00
                 2019    3              nan
                         6              nan
                         9              nan
                         12       29,327.00
                 2020    3              nan
                         6              nan
                         9              nan
                         12       29,850.00
                 2021    3              nan
アシックス            1999    9              nan
                 2000    3         4,270.00
                         9         4,348.00
                 2001    3         4,273.00
                         9         4,233.00
                 2002    3         4,109.00
                         9         4,236.00
                 2003    3         4,132.00
Name: 期末従業員数, dtype: float64

In [10]:
# forward fill groupby firm level
df_2008["期末従業員数"] = df_2008.groupby(level=0)["期末従業員数"].fillna(method='ffill')

<ipython-input-10-4ee0692f63b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["期末従業員数"] = df_2008.groupby(level=0)["期末従業員数"].fillna(method='ffill')


In [11]:
df_2008.loc[pd.IndexSlice[["アサヒグループホールディングス", "アシックス"], :, :], :]["期末従業員数"].iloc[70:90]

企業名              決算期（年）  決算期（月）
アサヒグループホールディングス  2018    6        30,864.00
                         9        30,864.00
                         12       28,055.00
                 2019    3        28,055.00
                         6        28,055.00
                         9        28,055.00
                         12       29,327.00
                 2020    3        29,327.00
                         6        29,327.00
                         9        29,327.00
                         12       29,850.00
                 2021    3        29,850.00
アシックス            1999    9              nan
                 2000    3         4,270.00
                         9         4,348.00
                 2001    3         4,273.00
                         9         4,233.00
                 2002    3         4,109.00
                         9         4,236.00
                 2003    3         4,132.00
Name: 期末従業員数, dtype: float64

* 資本的支出

欠損はほとんど半期報告のため。  
半期の値を2で割って前四半期に割り当てる。  
1年報告しかしていないなら4で割って各四半期に割り当てる。

まずは資本的支出の欠損パターンを確認

In [12]:
# check missing patterns
df_2008_capxisna = df_2008.loc[pd.IndexSlice[:, 2008:, 3:], :][
    df_2008.loc[pd.IndexSlice[:, 2008:, 3:], :]["資本的支出"].isna()
]

df_2008_capxisna.index.get_level_values("決算期（月）").unique()

Int64Index([9, 3, 6, 12], dtype='int64', name='決算期（月）')

どの四半期も欠損し得る

In [13]:
df_2008_capxisna["資本的支出"]

企業名          決算期（年）  決算期（月）
あい　ホールディングス  2011    9        nan
             2012    3        nan
                     9        nan
             2013    3        nan
                     9        nan
                               ..
ｅＢＡＳＥ        2018    12       nan
             2019    6        nan
                     12       nan
             2020    6        nan
                     12       nan
Name: 資本的支出, Length: 22747, dtype: float64

In [14]:
# firm missing Q1
missQ1 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [6]], :]["資本的支出"].index.get_level_values(0).unique()
missQ1

Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'きんでん', 'ぐるなび', 'さくらインターネット', 'たけびし',
       'なとり', 'はせがわ',
       ...
       'ＴＯＹＯ　ＴＩＲＥ', 'ＴＰＲ', 'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス',
       'ＷＤＢホールディングス', 'ＷＯＷＯＷ', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1082)

In [15]:
# firm missing Q2
missQ2 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [9]], :]["資本的支出"].index.get_level_values(0).unique()
missQ2

Index(['あい　ホールディングス', 'いであ', 'きちりホールディングス', 'ひらまつ', 'アイティメディア', 'アイフィスジャパン',
       'アイ・エス・ビー', 'アイ・オー・データ機器', 'アウトソーシング', 'アグロ　カネショウ',
       ...
       'ＧＭＯペパボ', 'Ｉ－ＰＥＸ', 'ＩＮＰＥＸ', 'ＪＵＫＩ', 'ＫＮＴ－ＣＴホールディングス', 'ＭｏｎｏｔａＲＯ', 'ＮＪＳ',
       'ＳＢＳホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名', length=194)

In [16]:
# firm missing Q3
missQ3 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [12]], :]["資本的支出"].index.get_level_values(0).unique()
missQ3

Index(['あみやき亭', 'あらた', 'いすゞ自動車', 'いなげや', 'きんでん', 'ぐるなび', 'さくらインターネット', 'たけびし',
       'なとり', 'はせがわ',
       ...
       'ＴＯＹＯ　ＴＩＲＥ', 'ＴＰＲ', 'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス',
       'ＷＤＢホールディングス', 'ＷＯＷＯＷ', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1074)

In [17]:
# firm missing Q4
missQ4 = df_2008_capxisna.loc[pd.IndexSlice[:, :, [3]], :]["資本的支出"].index.get_level_values(0).unique()
missQ4

Index(['あい　ホールディングス', 'いであ', 'きちりホールディングス', 'アイフィスジャパン', 'アイ・エス・ビー',
       'アイ・オー・データ機器', 'アウトソーシング', 'アグロ　カネショウ', 'アゴーラ　ホスピタリティー　グ',
       'アサヒグループホールディングス',
       ...
       'ＧＭＯペパボ', 'Ｉ－ＰＥＸ', 'ＩＮＰＥＸ', 'ＪＵＫＩ', 'ＫＮＴ－ＣＴホールディングス', 'ＭｏｎｏｔａＲＯ', 'ＮＪＳ',
       'ＳＢＳホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名', length=194)

In [18]:
missQ1 & missQ2 & missQ3 & missQ4

Index(['アシックス', 'インテージホールディングス', 'エス・サイエンス', 'オムロン', 'クラレ', 'コナミホールディングス',
       'サイバネットシステム', 'サイバーコム', 'サカタインクス', 'システムソフト', 'ソディック', 'ソリトンシステムズ',
       'デジタルガレージ', 'トラスコ中山', 'フルキャストホールディングス', 'マキタ', 'メック', 'ユニカフェ',
       'ユニ・チャーム', 'ローランド　ディー．ジー．', 'ヴィンクス', '井関農機', '富士ソフト', '小林製薬', '市光工業',
       '手間いらず', '日本ペイントホールディングス', '日本商業開発', '日機装', '日清紡ホールディングス', '日華化学',
       '星光ＰＭＣ', '東京応化工業', '東洋インキＳＣホールディングス', '江崎グリコ', '資生堂', '電通国際情報サービス',
       '鳥居薬品', 'ＩＮＰＥＸ', 'ＫＮＴ－ＣＴホールディングス', 'ＴＨＫ', 'ＴＯＹＯ　ＴＩＲＥ'],
      dtype='object', name='企業名')

In [19]:
df_2008.loc[pd.IndexSlice["日本商業開発", :, :], :]["資本的支出"]

企業名     決算期（年）  決算期（月）
日本商業開発  2003    3                   nan
        2004    3                   nan
        2005    3                   nan
        2006    3        107,000,000.00
        2007    3        150,000,000.00
                9                   nan
                12                  nan
        2008    3                   nan
                6                   nan
                9         52,000,000.00
                12        53,000,000.00
        2009    3         53,000,000.00
                6                   nan
                9                   nan
                12                  nan
        2010    3         10,000,000.00
                6         20,000,000.00
                9         20,000,000.00
                12        20,000,000.00
        2011    3         20,000,000.00
                6                   nan
                9                   nan
                12                  nan
        2012    3                   nan
                6

めんどくせぇ、でもどちらにせよ後の発表された資本的支出の額に前の欠損値分の資本的支出が含まれていて、欠損の分を問う分割して配分すればおっけー。

全部欠損している企業はもうどうしようもない、サンプルから落とすしかない。

In [20]:
# 欠損 50% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.5
])

7

In [21]:
# 欠損 40% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.4
])

16

In [22]:
# 欠損 30% 以上
len(df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0)[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=0) >= 0.3
])

1082

In [23]:
df_2008.loc[pd.IndexSlice["システムソフト", 2008:, :], :]["資本的支出"]

企業名      決算期（年）  決算期（月）
システムソフト  2008    3          1,000,000.00
                 6          3,000,000.00
                 9          5,000,000.00
                 12                  nan
         2009    3                   nan
                 6                   nan
                 9                   nan
                 12        12,000,000.00
         2010    3         12,000,000.00
                 6         12,000,000.00
                 9         12,000,000.00
                 12                  nan
         2011    3                   nan
                 6                   nan
                 9        944,000,000.00
                 12                  nan
         2012    3          1,000,000.00
                 6                   nan
                 9         21,000,000.00
                 12                  nan
         2013    3         26,000,000.00
                 6                   nan
                 9         54,000,000.00
                 12              

欠損率が大きくても、これなら埋めれる。(会計年度内最低1回報告している)

In [24]:
df_2008.loc[pd.IndexSlice["マキタ", 2008:, :], :]["資本的支出"]

企業名  決算期（年）  決算期（月）
マキタ  2008    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2009    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2010    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2011    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2012    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2013    3                      nan
             6                      nan
             9                      nan
             12                     nan
     2014    3      

このような欠損(１年に渡って欠損している)は救いようがない

チェックすべき(除外すべき)なのは、会計年度1年を通して欠損している企業かどうか consecutive nan

levelを会計年度にするためにmultiindexしなおす

In [25]:
# change index to 会計年度, 四半期
df_2008["会計年度"] = df_2008["決算期"].apply(lambda x: int(x[:4])-1 if int(x[-2:]) <= 3 else int(x[:4])) # 1, 2, 3月なら西暦-1
df_2008["四半期"] = df_2008["決算期"].apply(lambda x: 
                            "Q1" if int(x[-2:]) ==  6 else (
                            "Q2" if int(x[-2:]) ==  9 else (
                            "Q3" if int(x[-2:]) == 12 else (
                            "Q4" if int(x[-2:]) ==  3 else (
                            "NonMar" + x[-2:]
                            ))))
                           )

<ipython-input-25-6295ca30ea3d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["会計年度"] = df_2008["決算期"].apply(lambda x: int(x[:4])-1 if int(x[-2:]) <= 3 else int(x[:4])) # 1, 2, 3月なら西暦-1
<ipython-input-25-6295ca30ea3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2008["四半期"] = df_2008["決算期"].apply(lambda x:


In [26]:
# set multiindex ["企業名", "会計年度", "四半期"]
df_2008.reset_index(inplace=True)
df_2008.set_index(["企業名", "会計年度", "四半期"], inplace=True)

In [27]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1])[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1]) == 1
]

企業名          会計年度
エス・サイエンス     2017   1.00
             2019   1.00
オムロン         2009   1.00
             2012   1.00
             2014   1.00
             2015   1.00
             2019   1.00
コナミホールディングス  2008   1.00
             2010   1.00
             2011   1.00
             2013   1.00
             2014   1.00
             2015   1.00
             2016   1.00
マキタ          2008   1.00
             2009   1.00
             2010   1.00
             2011   1.00
             2012   1.00
             2013   1.00
             2014   1.00
             2015   1.00
             2016   1.00
手間いらず        2014   1.00
             2019   1.00
日本商業開発       2011   1.00
             2012   1.00
Name: 資本的支出, dtype: float64

2008年度以降、1会計年間に１つも資本的支出が観測されていないのは27年

In [28]:
whole_miss_capx = df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1])[
    df_2008.loc[pd.IndexSlice[:, 2008:, :], :]["資本的支出"].isna().mean(level=[0, 1]) == 1
].index.get_level_values(0).unique()

whole_miss_capx

Index(['エス・サイエンス', 'オムロン', 'コナミホールディングス', 'マキタ', '手間いらず', '日本商業開発'], dtype='object', name='企業名')

この企業は会計年度1年間丸ごと欠損しているのでサンプルから drop していい

欠損パターンは("Q1", "Q3"), ("Q2", "Q4"), ("Q1", "Q2", "Q3"), ("Q2", "Q3", "Q4"), ("Q1", "Q2", "Q4), ("Q1", "Q3", "Q4)か、1つだけ、もしくは全部(これはdropする)

In [29]:
# previous_fiscal_year = None
# # for i in df_2008.index.get_level_values(0).unique():
# for i in df_2008.index.get_level_values(0).unique():
# #     print(i)
#     for j in df_2008.loc[pd.IndexSlice[i, 2008:, :], :].index.get_level_values(1).unique():
#         temp_df = df_2008.loc[pd.IndexSlice[i, j, :], :]["資本的支出"]
# #         print(temp_df)
#         if temp_df.isna().sum() > 0:
# #             print("missing")
#             missing_type = list(temp_df[temp_df.isna()].index.get_level_values(2).values)
# #             print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#             if missing_type == ["Q1", "Q3"]:
#                 pass
#             elif missing_type == ["Q2", "Q4"]:
#                 pass
#             elif missing_type == ["Q1", "Q2", "Q3"]:
#                 print(i)
#                 print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#                 pass
#             else:
#                 print(i)
#                 print("missing type: ", temp_df[temp_df.isna()].index.get_level_values(2).values)
#                 pass
#         else:
# #             print("no missing")
#             pass

In [30]:
df_2008.loc[pd.IndexSlice["オムロン", :, :], :]["資本的支出"].iloc[0:100]

企業名   会計年度  四半期
オムロン  1999  Q2                  nan
            Q4    31,146,000,000.00
      2000  Q2                  nan
            Q4    37,583,000,000.00
      2001  Q2                  nan
            Q4    38,896,000,000.00
      2002  Q2                  nan
            Q4    34,454,000,000.00
      2003  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    38,115,000,000.00
      2004  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    38,579,000,000.00
      2005  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    40,560,000,000.00
      2006  Q1                  nan
            Q2                  nan
            Q3                  nan
            Q4    44,689,000,000.00
      2007  Q1                  nan
            Q2                  nan
            Q3                  nan
            

In [31]:
df_2008.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"].iloc[0:100]

企業名      会計年度  四半期
システムソフト  1999  Q2               nan
               Q4    143,000,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4               nan
         2006  Q1      9,000,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

In [32]:
df_2008.loc[pd.IndexSlice["ＴＨＫ", :, :], :]["資本的支出"].iloc[0:100]

企業名  会計年度  四半期
ＴＨＫ  1999  Q2                  nan
           Q4     4,857,000,000.00
     2000  Q2     4,009,000,000.00
           Q4    11,076,000,000.00
     2001  Q2     7,647,000,000.00
           Q4     9,226,000,000.00
     2002  Q2     2,578,000,000.00
           Q4     4,759,000,000.00
     2003  Q1     1,624,000,000.00
           Q2     2,500,000,000.00
           Q3     3,802,000,000.00
           Q4     5,105,000,000.00
     2004  Q1     1,774,000,000.00
           Q2     2,663,000,000.00
           Q3     6,001,000,000.00
           Q4     6,963,000,000.00
     2005  Q1     2,712,000,000.00
           Q2     6,989,000,000.00
           Q3     8,524,000,000.00
           Q4    12,520,000,000.00
     2006  Q1     2,601,000,000.00
           Q2     5,386,000,000.00
           Q3     8,845,000,000.00
           Q4    12,848,000,000.00
     2007  Q1    15,305,000,000.00
           Q2     8,995,000,000.00
           Q3    14,327,000,000.00
           Q4    19,618,000,000.00
     

Q4は本決算だから累計かも? --> Q4 < Q2 とかあるので大丈夫なはず

やっぱり四半期報告である以上、報告していない分は時期に含まれている気がするから、年度またいだとしても(つまりQ4の欠損値に対して)次年度を分割して穴埋めする方針で。

In [33]:
# 丸ごと1年欠損している企業をdrop
df_2008 = df_2008.loc[pd.IndexSlice[(recorded_firm_from_2008 - set(whole_miss_capx)), :, :], :]

In [34]:
df_2008.index.get_level_values(0).unique()

Index(['あい　ホールディングス', 'あみやき亭', 'あらた', 'いすゞ自動車', 'いであ', 'いなげや', 'かどや製油',
       'きちりホールディングス', 'きんでん', 'ぐるなび',
       ...
       'ＴＹＫ', 'ＵＡＣＪ', 'ＵＴグループ', 'ＶＴホールディングス', 'ＷＤＢホールディングス', 'ＷＯＷ　ＷＯＲＬＤ',
       'ＷＯＷＯＷ', 'Ｚホールディングス', 'ＺＯＺＯ', 'ｅＢＡＳＥ'],
      dtype='object', name='企業名', length=1430)

ちょうど5の倍数、VRPで割り切れる  
"システムソフト"の欠損が一番多くてバリエーションが多い?っぽいからこいつをうまくできればおっけー

In [35]:
df_2008_copy = df_2008.copy()

In [36]:
df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"]

企業名      会計年度  四半期
システムソフト  1999  Q2               nan
               Q4    143,000,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4               nan
         2006  Q1      9,000,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

In [39]:
isna_series = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].isna()

for i in range(len(isna_series)-1):
    if (isna_series.iloc[i] == True) & (isna_series.iloc[i+1] == False): 
        # i が nan, i+1 が nan じゃないとき、
        ind_non_nan = df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :].index[i+1]
        print("index of record be divided: ", ind_non_nan)
        print(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i], df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"][i+1])
        num_nan = 0
        ind_nan_list = []
        while isna_series.iloc[i - num_nan]:
            # nan がどのくらい続いていたのかカウントして、
            num_nan += 1
            ind_nan_list.append(df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], "資本的支出"].index[i+1 - num_nan])
        print("num nan: ", num_nan)
        print("index of nan: ", ind_nan_list)
        # nan じゃない i+1 を num_nan + 1の分だけ割っておいて、
        divided_value = (df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] / (num_nan+1))
        print("divided value: ", divided_value)
        df_2008_copy.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] = divided_value
        for ind_nan in ind_nan_list:
            # nan にも同じ割った値を代入
            df_2008_copy.loc[pd.IndexSlice[ind_nan[0], ind_nan[1], ind_nan[2]], "資本的支出"] = divided_value

index of record be divided:  ('システムソフト', 1999, 'Q4')
nan 143000000.0
num nan:  1
index of nan:  [('システムソフト', 1999, 'Q2')]
divided value:  71500000.0
index of record be divided:  ('システムソフト', 2006, 'Q1')
nan 9000000.0
num nan:  1
index of nan:  [('システムソフト', 2005, 'Q4')]
divided value:  4500000.0
index of record be divided:  ('システムソフト', 2009, 'Q3')
nan 12000000.0
num nan:  4
index of nan:  [('システムソフト', 2009, 'Q2'), ('システムソフト', 2009, 'Q1'), ('システムソフト', 2008, 'Q4'), ('システムソフト', 2008, 'Q3')]
divided value:  2400000.0
index of record be divided:  ('システムソフト', 2011, 'Q2')
nan 944000000.0
num nan:  3
index of nan:  [('システムソフト', 2011, 'Q1'), ('システムソフト', 2010, 'Q4'), ('システムソフト', 2010, 'Q3')]
divided value:  236000000.0
index of record be divided:  ('システムソフト', 2011, 'Q4')
nan 1000000.0
num nan:  1
index of nan:  [('システムソフト', 2011, 'Q3')]
divided value:  500000.0
index of record be divided:  ('システムソフト', 2012, 'Q2')
nan 21000000.0
num nan:  1
index of nan:  [('システムソフト', 2012, 'Q1')]
divided value:  1

In [40]:
df_2008_copy.loc[pd.IndexSlice["システムソフト", :, :], :]["資本的支出"]

企業名      会計年度  四半期
システムソフト  1999  Q2     71,500,000.00
               Q4     71,500,000.00
         2000  Q2      7,000,000.00
               Q4     20,000,000.00
         2001  Q2     26,000,000.00
               Q4     34,000,000.00
         2002  Q2     65,000,000.00
               Q4    101,000,000.00
         2003  Q1      2,000,000.00
               Q2      6,000,000.00
               Q3      5,000,000.00
               Q4      7,000,000.00
         2004  Q1      1,000,000.00
               Q2      5,000,000.00
               Q3      8,000,000.00
               Q4     12,000,000.00
         2005  Q1      8,000,000.00
               Q2      9,000,000.00
               Q3     10,000,000.00
               Q4      4,500,000.00
         2006  Q1      4,500,000.00
               Q2     26,000,000.00
               Q3     24,000,000.00
               Q4     31,000,000.00
         2007  Q1     36,000,000.00
               Q2     44,000,000.00
               Q3      1,000,000.00
         

できた!これを全社でループする

In [41]:
# fill nan (devide after equal) for "資本的支出"

for firm in df_2008.index.get_level_values(0).unique():
    isna_series = df_2008.loc[pd.IndexSlice[firm, :, :], "資本的支出"].isna()

    for i in range(len(isna_series)-1):
        if (isna_series.iloc[i] == True) & (isna_series.iloc[i+1] == False): 
            # i が nan, i+1 が nan じゃないとき、
            ind_non_nan = df_2008.loc[pd.IndexSlice[firm, :, :], :].index[i+1]

            num_nan = 0
            ind_nan_list = []
            while isna_series.iloc[i - num_nan]:
                # nan がどのくらい続いていたのかカウントして、
                num_nan += 1
                ind_nan_list.append(df_2008.loc[pd.IndexSlice[firm, :, :], "資本的支出"].index[i+1 - num_nan])
            # nan じゃない i+1 を num_nan + 1の分だけ割っておいて、
            divided_value = (df_2008.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] / (num_nan+1))
            df_2008.loc[pd.IndexSlice[ind_non_nan[0], ind_non_nan[1], ind_non_nan[2]], "資本的支出"] = divided_value

            for ind_nan in ind_nan_list:
                # nan にも同じ割った値を代入
                df_2008.loc[pd.IndexSlice[ind_nan[0], ind_nan[1], ind_nan[2]], "資本的支出"] = divided_value
#     print(firm, " done")

あい　ホールディングス  done
あみやき亭  done
あらた  done
いすゞ自動車  done
いであ  done
いなげや  done
かどや製油  done
きちりホールディングス  done
きんでん  done
ぐるなび  done
さくらインターネット  done
たけびし  done
なとり  done
はせがわ  done
はるやまホールディングス  done
ひらまつ  done
ぴあ  done
やまや  done
りらいあコミュニケーションズ  done
わかもと製薬  done
アイカ工業  done
アイコム  done
アイシン  done
アイダエンジニアリング  done
アイチコーポレーション  done
アイティフォー  done
アイティメディア  done
アイネス  done
アイネット  done
アイフィスジャパン  done
アイホン  done
アイロムグループ  done
アイ・エス・ビー  done
アイ・オー・データ機器  done
アウトソーシング  done
アキレス  done
アクシアル　リテイリング  done
アクセル  done
アグロ　カネショウ  done
アゴーラ　ホスピタリティー　グ  done
アサヒグループホールディングス  done
アシックス  done
アジアパイルホールディングス  done
アステラス製薬  done
アステリア  done
アズビル  done
アズワン  done
アツギ  done
アドウェイズ  done
アドソル日進  done
アドバネクス  done
アドバンテスト  done
アドバンテッジリスクマネジメン  done
アドヴァングループ  done
アネスト岩田  done
アバント  done
アマダ  done
アマノ  done
アミューズ  done
アリアケジャパン  done
アルインコ  done
アルコニックス  done
アルゴグラフィックス  done
アルバック  done
アルビス  done
アルファ  done
アルファシステムズ  done
アルフレッサ　ホールディングス  done
アルプスアルパイン  done
アルプス技研  done
アルプス物流  done
アルペン  done
アンリツ  do

In [46]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"].isna().sum()

13

In [50]:
df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"][df_2008.loc[pd.IndexSlice[:, 2008:, :], "資本的支出"].isna()]

企業名              会計年度  四半期
あい　ホールディングス      2020  Q4    nan
インテージホールディングス    2020  Q4    nan
オルバヘルスケアホールディング  2020  Q4    nan
ディア・ライフ          2020  Q3    nan
                       Q4    nan
トーア紡コーポレーション     2020  Q4    nan
パイロットコーポレーション    2020  Q4    nan
フルキャストホールディングス   2020  Q4    nan
メック              2020  Q4    nan
日本エスコン           2020  Q4    nan
鈴木               2020  Q4    nan
ＣＩＪ              2020  Q4    nan
ＩＮＰＥＸ            2020  Q4    nan
Name: 資本的支出, dtype: float64

最後の期2020Q4が欠損していたらどうしようもないが、ぶっちゃけテスト期間になるので欠損したままで良い。


とりあえずここまでで一回保存

In [51]:
df_2008.to_csv("./../data/processed/tse1_2008_capx_labor_filled.csv")

In [53]:
df_2008 = pd.read_csv("./../data/processed/tse1_2008_capx_labor_filled.csv", index_col=[0, 1, 2])

* 四半期EPS

In [56]:
df_2008.isna().mean()

決算期（年）                      0.00
決算期（月）                      0.00
決算期                         0.00
連結基準フラグ                     0.00
決算種別フラグ                     0.00
決算発表日                       0.02
事業年度開始年月日［３ヵ月］              0.28
事業年度終了年月日                   0.00
１株当たり利益［３ヵ月］                0.30
１株当たり利益［累計］                 0.03
当期純利益（連結）［累計］               0.47
【ＱＴＲ】当期利益                   0.13
棚卸資産                        0.05
受取手形・売掛金／売掛金及びその他の短期債権      0.02
資本的支出                       0.00
売上総利益［累計］                   0.02
【ＱＴＲ】売上総利益                  0.14
販売費及び一般管理費［累計］              0.03
法人税等［累計］                    0.02
法人税・住民税及び事業税合計／当期税金費用［累計］   0.20
税金等調整前当期純利益［累計］             0.12
税金等調整前当期利益［累計］              0.05
【ＱＴＲ】税金等調整前当期利益             0.16
１株当たり税引前利益                  0.11
売上高・営業収益［累計］                0.01
【ＱＴＲ】売上高・営業収益               0.11
１株当たり売上高                    0.12
期末従業員数                      0.00
純資産合計／資本合計                  0.24
【ＱＴＲ】純資産                    0.25
１株当たり純資産  

欠損処理後、最後に期間を2008Q1~にカットする

## ファンダメンタル会計変数の生成

## Data Preprocessing

データのスケーリング

$$\begin{eqnarray}
    INV &=& \frac{棚卸資産}{普通株式の期中加重平均株式数} \\
    AR &=& \frac{売掛金}{普通株式の期中加重平均株式数} \\
    CAPX &=& \frac{資本的支出}{普通株式の期中加重平均株式数} \\
    GM &=& \frac{売上総利益}{普通株式の期中加重平均株式数} \\
    SA &=& \frac{販売費及び一般管理費}{普通株式の期中加重平均株式数} \\
    ETR &=& 実効税率 \\
    LF &=& \log{労働力} \\
\end{eqnarray}$$